In [ ]:
import pandas as pd
import zstandard
import io
import wget
import os

# Dataset download from pushshift as JSON

In [ ]:
def check_if_ca(line):
    keywords = ["cultural appropriation", "cultural misappropriation", "cultural insensitivity", "cultural stereotyping",
            "cultural exploitation", "cultural misrepresentation", "traditional symbol", "traditional clothing",
            "linguistic appropriation", "language appropriation", "religious appropriation", "cultural commodification",
            "artifact appropriation"]
    if any(ext in line.lower() for ext in keywords):
        return True
    else:
        return False

In [ ]:
for i in range(1,4):
    month = f'{i}'
    if i < 10:
        month = f'0{i}'
    url = f'https://files.pushshift.io/reddit/comments/RC_2022-{month}.zst'
    filename = wget.download(url)
    with open('comments_2022.json', 'a+') as out:
        with open(filename, 'rb') as fh:
            dctx = zstandard.ZstdDecompressor(max_window_size=2147483648)
            stream_reader = dctx.stream_reader(fh)
            text_stream = io.TextIOWrapper(stream_reader, encoding='utf-8')
            for line in text_stream:
                if check_if_ca(line):
                    out.write(line)
    os.remove(filename)

In [ ]:
comments = pd.read_json('comments_2022.json', lines=True)
comments = comments[['id', 'parent_id', 'author','subreddit','body']]
comments.to_csv('comments_2022.csv', index=False)